In [1]:
#custom libraries
from data_import import *
from data_preprocess import *
from data_preprocess_BERT import *

In [2]:
#set filepath constants
NLP_TOPIC_RECOGNITION_FILEPATH = '../NLP Topic Recognition'
#NLP_MODEL_FILEPATH = '../NLP Model Training/model_100K_glove'
NLP_MODEL_FILEPATH = '../NLP Model Training/model_100K_glove_91'
TEXTUAL_RAW_DATA_FILEPATH = '../Data/textual_raw'

## Glove 0.91
OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_91_glove_filtered_full.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_91_filtered_full_user_averaged.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_91_filtered_full_user_weights_user_averaged.csv"

## Glove standar
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_full_user_weights_user_averaged.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_full_user_weights.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_2g_user_weights.csv"

## BERT
# OUTPUT_DATASET_FILEPATH_PREV = "Dataset_sentiment_BERT_trainable.csv"
# OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_BERT_trainable_full.csv"

In [3]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(NLP_TOPIC_RECOGNITION_FILEPATH)
from topic_recognition import *
#from NLP_class import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [ ]:
from tensorflow import keras
model = keras.models.load_model(NLP_MODEL_FILEPATH)

In [ ]:
# returns a dictionary of the form {'username': (sentiment_score_sum, occurances)}
def get_users(paths):
    df = pd.DataFrame()
    for path in paths:
        text_data = preprocess_textual(path)
        df = pd.concat([df, text_data])
    
    #take the set of all usernames
    ini_set = set(df['username'].values)
    #initialized as neutral (0.5) and 1 occurance
    users = dict.fromkeys(ini_set, (0.5,1))
    return users

#updates username's sentiment score values after recieving username's tweet
def user_update(username, sentiment):
    #update 
    (s,t) = users[username]
    (s_new, t_new) = (s + sentiment, t + 1)
    users[username] = (s_new, t_new)
    
    #sentiment calculation sentiment_weighted has range(-1, 1)
    #the extreme values are seen when a positive (on average) user tweets something really negative and the converse
    sentiment_weighted = sentiment - s/t
    return sentiment_weighted

In [ ]:
#Create a bert_preprocess object that automatically sets tokenizers and pretrained bert model classifier
bert = BERT_preprocess()

In [ ]:
# lol = text.df[:100]
# lol['sentiment']=bert.predict(list(lol['text'].values))
# pd.options.display.max_colwidth = 400
# print(lol[['sentiment','text']])
# lol = text.df
# list(text.df['text'].values)
# start = timer()
# lol.text.map(lambda x: bert.predict([x]))
# print(timer() - start)

# Sentiment analysis on Tweets

## BERT

In [ ]:
#fetch btc price data
btc_data = preprocess_BTC()

#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

#initialize username dictionary for user wieghted sentiment scores
#users = get_users(paths)

In [ ]:
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]
print(paths)

In [ ]:
i = 1

# final dataframe (initialize or load)
# df = pd.DataFrame()
df = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)
print(df)
start = timer()
for path in paths:
#     if i<47:
#         i+=1
#         continue
    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    #print(text.df.shape[0])
    #perform some spam_filtering on the current dataset
    text.spam_filtering()
    #print(text.df.shape[0])
    
    #calculate the sentiment score for every tweet in the current dataset
    text.df['sentiment'] = bert.predict(list(text.df['text'].values))
    
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    #print(text.df)
   
    #get the mean and median sentiment per hour
    timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
    timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    #if mismatching sizes disregard current dataset
    if len(timetable.index) != len(btc_connected.index):
        continue
    #otherwise combine them into a single dataset 
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['Open']], axis = 1)
    #print(timetable)
    
    df = pd.concat([df, timetable], axis = 0)   
    print(df)
    
    #save final dataset
    df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


In [ ]:
df.to_csv(OUTPUT_DATASET_FILEPATH)

## Plain Glove

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [ ]:
#fetch btc price data
btc_data = preprocess_BTC()

#itterate folder for all raw textual data
# txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
# paths = [x for x in txt_folder]

#initialize username dictionary for user wieghted sentiment scores
#users = get_users(paths)

In [4]:
#itterate folder for all raw textual data
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]

i = 1

# final dataframe
df = pd.DataFrame()
weighted = False

start = timer()
for path in paths:

    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
#     old approach
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)
    
    #initialize an NLP_preprocess object for the current dataset
    text = NLP_preprocess(text_data)
    
    #print(text.df.shape[0])
    text.flag_users()
    
    
    #basic nlp preprocessing (rm stopwords, rm punctuation etc.)
    text.preprocess_data()
    #perform some spam filtering
    text.spam_filtering()
    
    #print(text.df.shape[0])
    
    #import default tokenizer for the model
    text.import_tokenizer()
    #tokenize and pad text sequences of the dataset
    df_padded = text.tokenize_and_pad(train = False)
        
    #apply the model on the padded sequences to get sentiment
    text.df["sentiment"] = (model.predict(df_padded))
    #print(text.df.index)
    
#     for r in np.random.randint(0,text.df.shape[0],10):
#         print(text.df.iloc[r]['text'], text.df.iloc[r]['sentiment'])


    #try: weight messeges by the average optimism/pessimism of each user
    #this might have an impact because there are about 5.5 messages per user,
    #so there is some history about them
    if weighted:
        text.df['weighted'] = text.df.index.map(lambda x: user_update(text.df.loc[x]['username'],  text.df.loc[x]['sentiment'])) 
    #print(text.df)
    #add an "hour" column that contains the datetime in hourly resolution
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    
    #old
#     timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    

    #and then get the mean and median weighted sentiment per hour
    if weighted:
        #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
        user_averaged = text.df.groupby(['hour', 'username'], as_index=False).weighted.apply(np.mean)
        timetable = pd.DataFrame(user_averaged.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
        timetable2 = pd.DataFrame(user_averaged.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
    else:
        #we can eliminate the big effect of users that sent multiple messages by first averaging per user per hour
        user_averaged = text.df.groupby(['hour', 'username'], as_index=False).sentiment.apply(np.mean)
#         timetable = pd.DataFrame(user_averaged.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
#         timetable2 = pd.DataFrame(user_averaged.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
        
        timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
        timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    #get the mean and median weighted sentiment per hour
#     timetable = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.mean).values, columns = ["Bitcoin"])
#     timetable2 = pd.DataFrame(text.df.groupby(['hour']).weighted.apply(np.median).values, columns = ["Bitcoin_median"])
     
    #sync the price and sentiment dataset so that on each row price follows the sentiment
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    #if mismatching sizes disregard current dataset
    if len(timetable.index) != len(btc_connected.index):
        continue
    #otherwise combine them into a single dataset 
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['open'],btc_connected['close']], axis = 1)
    print(timetable)
    
    df = pd.concat([df, timetable], axis = 0)   
    #print(df)

#save final dataset
df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


Iterration 1 of 65
Duplicate messages in a day
"Bitcoin Tumbles Below $4000 - Down 21% From Record High 
"North Korean hackers target bitcoin, Facebook bans monetization of offensive content and more on #CrunchReport wit… https://twitter.com/i/web/status/908116090659651584 …"
"JPMorgan CEO Says That Bitcoin Is a “Fraud” http://ift.tt/2w8a1E7"
"http://bitzfree.com - Free Bitcoin Cloud Mining http://bitzfree.com"
"North Korea Is Dodging Sanctions With a Secret Bitcoin Stash - Bloomberg https://www.bloomberg.com/news/articles/2017-09-11/north-korea-hackers-step-up-bitcoin-attacks-amid-rising-tensions …"
"Bitcoin is falling and I'm pretty sure most people have started dumping :("
"Bitcoin is the future"
"FreeBitco.in - Free Bitcoin Wallet, Faucet, Lottery and Dice! http://freebitco.in/?r=6348901"
"Argentina Jumps on Bitcoin ATM Bandwagon with 200 Expected in October https://cointelegraph.com/news/argentina-jumps-on-bitcoin-atm-bandwagon-with-200-expected-in-october … via @Cointelegraph"
"R

NameError: name 'model' is not defined

# Evaluation and Testing (not important for now)

In [ ]:
df2 = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)

In [ ]:
DATASET_FILEPATH = "./Dataset_sentiment_100K_glove_filtered_2g.csv"
df2 = pd.read_csv(OUTPUT_DATASET_FILEPATH, index_col = 0)
fig = plt.figure(figsize=(18, 10))
plt.plot(range(0,100), -1*df2['Bitcoin'].values[:100])

In [ ]:
fig = plt.figure(figsize=(18, 10))
x = -df2['Bitcoin'][100:200]
x = x/x[0]
y = df2['Open'][100:200]
y = y/y[0]
plt.plot(range(0,100), x)
plt.plot(range(0,100), y)


In [ ]:
fig = plt.figure(figsize=(18, 10))
plt.plot(range(0,100), df.pct_change()['Open'].values[100:200])

In [ ]:
import random
i = random.randint(0,len(text_data.index))
print(text_data.loc[i]['text'])
print(text_data.loc[i]['sentiment'])

In [ ]:
#BTC price data of the next hour standarized
Y = pd.DataFrame(df['Open']).astype(float)
#Y = dataset_standarization(Y)

#Textual data ('Bitcoin' topic popularity) of the last hour standarized
X = pd.DataFrame(df['Bitcoin']).astype(float)
#X = dataset_standarization(X)
Y.index = X.index

X = sm.add_constant(X)

model = sm.OLS(Y, X, missing = 'drop').fit()

In [ ]:
model.summary()